# Single Machine Infinite Bus (SMIB) - Network to Machine Reference Frame Conversion
#### Khaled Aleikish
#### University of South-Eastern Norway
#### Porsgrunn, April 2023

## Preliminaries
### Packages

In [1]:
import math
import cmath
import numpy as np
from scipy import linalg

#### Utility functions

In [2]:
def cis(x):
    return complex(math.cos(x), math.sin(x))

def PrintPolar(txt, x, f1, f2):
    return print(txt+f" = {abs(x):.{f1}f}∠{cmath.phase(x)*180/cmath.pi:.{f2}f}°")

# TET4180 SMIB

![TET4180 SMIB](TET4180_SMIB.png)

## System Parameters

#### Network parameters

In [3]:
fn = 50 # Nominal frequency, Hz
wn = 2*math.pi*fn # Nominal frequency, rad/s
IB_V = 220 # Infinite busbar voltage, kV

#### Generator parameters

In [4]:
SgN = 50 # Rated apparent power, MVA
UgN = 10 # Rated voltage, kV

Xd = complex(0, 1.05) # d-axis synchronous reactance, pu
Xq = complex(0, 0.66) # q-axis synchronous reactance, pu

Xd_prime = complex(0, 0.328) # d-axis transient reactance, pu
# Xq_prime = ? # q-axis transient reactance, pu

Xd_double_prime = complex(0, 0.254) # d-axis subtransient reactance, pu
Xq_double_prime = complex(0, 0.273) # q-axis subtransient reactance, pu

Tdo_prime = 2.49 # Open-circuit d-axis transient time constants, s
# Tq0_prime = ? # Open-circuit q-axis transient time constants, s

Tdo_double_prime = 0.06 # Open-circuit d-axis subtransient time constant, s
Tqo_double_prime = 0.15 # Open-circuit q-axis subtransient time constant, s

Td_prime = Tdo_prime * (Xd_prime/Xd) # Short-circuit d-axis transient time constant, s
# Tq_prime = Tqo_prime * (Xq_prime/Xq) # Short-circuit q-axis transient time constant, s

Td_double_prime = Tdo_double_prime * (Xd_double_prime/Xd_prime) # Short-circuit d-axis subtransient time constant, s
# Tq_double_prime = Tqo_double_prime * (Xq_double_prime/Xq_prime) # Short-circuit q-axis subtransient time constant, s

Ra = 0 # Armature resistance, pu
H = 3.1 # Inertia constant, s or (MWs/MVA)
Xl = complex(0, 0.1) # Armature leakage reactance, pu

# Parameters are passed to functions in a list to avoid problems associated with global variables.
genPar = [Xd, Xq, Xd_prime, Xd_double_prime, Xq_double_prime, Tdo_prime, Tdo_double_prime, Tqo_double_prime, Td_prime, Td_double_prime]

#### Transformer parameters

In [5]:
Xt = complex(0, 0.1) # Transformer reactance, pu

# StN = 50 # Rated apparent power, MVA
# UtN = 10/245 # Rated voltage, kV

#### Local load

In [6]:
# Z_LL = float('inf') # Local load, Ohm
Z_LL = complex(2000,0) # Local load, Ohm

#### Line parameters
- Line resistance is neglected.

In [7]:
UN = 245 # The line voltage, kV
line_length = 250 # km
x = complex(0,0.4) # Ohm/km

#### Base quantities

In [8]:
S_base = SgN # Rated apparent power, MVA
V_base = UN # Base voltage, kV
Z_base = (V_base**2)/S_base # Base impedance, Ohm
I_base = V_base/(Z_base*cmath.sqrt(3))

print(f"Base impedance, Z_base = {Z_base:.1f} [Ohm]")
# print(f"Base current, I_base = {I_base:.1f} [Ohm]")

Base impedance, Z_base = 1200.5 [Ohm]


#### Initial operational parameters

In [9]:
P = 15 # MW
PF = 0.9 # Power factor lagging

## Initial power flow
- The inﬁnite bus voltage is set as the reference.
- The generator is over-excited at a known power factor PF = cos(φ).

In [10]:
IB_V_pu  = IB_V/V_base
PrintPolar("IB voltage, Vs", IB_V_pu, 3, 0)

X_l = (x*line_length)/Z_base  # Line reactance, pu
print(f"Line impedance, X_l = {X_l:.4f}")

Z_LL_pu = Z_LL/Z_base
print(f"Local load, R = {Z_LL_pu:.2f}")

IB voltage, Vs = 0.898∠0°
Line impedance, X_l = 0.0000+0.0833j
Local load, R = 1.67+0.00j


In [11]:
# # Test
# P_pu = 0.3
# PF = 0.9
# Q_pu = complex(0,P_pu * cmath.tan(cmath.acos(PF))) # Reactive power, MVAr
# S_pu = P_pu + Q_pu
# print(f"IB apparent power, S_pu = {S_pu:.3f}")
# print(f"Power factor angle = {-cmath.acos(PF).real * 180/math.pi}°")

In [12]:
P_pu = (P/S_base) # Real power, pu
Q_pu = complex(0,P_pu * cmath.tan(cmath.acos(PF))) # Reactive power, pu
S_pu = P_pu + Q_pu # Complex power, pu

print(f"IB Complex power, S_pu = {S_pu:.3f}")
print(f"IB apparent power, S_pu = {abs(S_pu):.3f}")

IB Complex power, S_pu = 0.300+0.145j
IB apparent power, S_pu = 0.333


In [13]:
I_line = np.conj(S_pu/IB_V_pu) # Check if the parentheses are correct
PrintPolar("Line current, I_line", I_line, 3, 1)

ΔV_line = I_line*X_l
PrintPolar("Voltage drop across the line, ΔV_line", ΔV_line, 4, 1)

V_Transformer = ΔV_line + IB_V_pu
PrintPolar("Load/Transformer voltage, V_Transformer", V_Transformer, 3, 2)

Line current, I_line = 0.371∠-25.8°
Voltage drop across the line, ΔV_line = 0.0309∠64.2°
Load/Transformer voltage, V_Transformer = 0.912∠1.75°


In [14]:
I_LL = V_Transformer/Z_LL_pu
PrintPolar("I_Local_Load", I_LL, 3, 2)

P_load = (V_Transformer * np.conj(I_LL)).real
print(f"Local load active Power = {P_load:.10f} [pu], or {P_load*SgN:.1f} MW")

I_Local_Load = 0.547∠1.75°
Local load active Power = 0.4991035842 [pu], or 25.0 MW


In [15]:
# Step 1: find I
I = I_line + I_LL
PrintPolar("Total generator current, I", I, 10, 10)

φ = cmath.phase(I)
print(f"\x1b[95mφ = {φ*180/cmath.pi:.2f}° : Angle between the infinite bus and the current\x1b[0m")

Total generator current, I = 0.8930483555∠-9.3509107724°
φ = -9.35° : Angle between the infinite bus and the current


In [16]:
ΔV_t = I*Xt
PrintPolar("Voltage drop across generator transformer, ΔV_t", ΔV_t, 4, 1)

# Step 2: find Vg
Vg = ΔV_t + V_Transformer
PrintPolar("Generator terminal voltage, Vg", Vg, 10, 10)
print(f"\x1b[95mLook at Vg angle!\x1b[0m")

PrintPolar("Transformer voltage, Vt", Vg-ΔV_t, 3, 2)

Voltage drop across generator transformer, ΔV_t = 0.0893∠80.6°
Generator terminal voltage, Vg = 0.9331790909∠7.1374426179°
Look at Vg angle!
Transformer voltage, Vt = 0.912∠1.75°


In [17]:
# Step 3: find 𝛿
EQ = Vg + I*Xq
𝛿 = cmath.phase(EQ)

# Step 4: find β so you can decompose the current to dq-frame
β = 𝛿 + -(φ)
PrintPolar("Internal emf, EQ", EQ, 3, 2)
print(f"\x1b[95m𝛿 = {𝛿*180/cmath.pi:.1f}° : Angle between Eq, or the q-axis, and IB\x1b[0m")
print(f"\x1b[95mβ = 𝛿 + -(φ) = {𝛿*180/cmath.pi:.1f}° + -({φ*180/cmath.pi:.1f}°) = {β*180/cmath.pi:.1f}° : Angle between Eq and I\x1b[0m")

Internal emf, EQ = 1.237∠34.32°
𝛿 = 34.3° : Angle between Eq, or the q-axis, and IB
β = 𝛿 + -(φ) = 34.3° + -(-9.4°) = 43.7° : Angle between Eq and I


In [18]:
print("\nThe steady-state define the initial conditions for the linear model. The necessary quantities are:\n")

Id = cmath.rect(abs(I)*math.sin(β),(𝛿 - cmath.pi/2)) # 𝛿 - 90°, phi must be in radian
Iq = cmath.rect(abs(I)*math.cos(β),𝛿) # 𝛿

PrintPolar("Id = |I|*sin(β)∠(𝛿 - π/2)", Id, 3, 1)
PrintPolar("Iq = |I|*cos(β)∠𝛿", Iq, 3, 1)

ΔV_Xd = Xd * Id
ΔV_Xq = Xq * Iq
PrintPolar("ΔV_Xd = Xd*Id", ΔV_Xd, 3, 1)
PrintPolar("ΔV_Xq = Xq*Id", ΔV_Xq, 3, 1)

Eq = Vg + ΔV_Xd + ΔV_Xq
PrintPolar("Eq", Eq, 2, 1)
print(f"The internal induced voltage = {abs(Eq)*UgN:.3f} kV")

Pg = (Vg * np.conj(I)).real
print(f"Generator active Power = {Pg:.10f} [pu], or {Pg*SgN:.1f} MW")

# or
# abs(V_Transformer)**2/Z_LL_pu + P_pu

Qg = (Vg * np.conj(I)).imag
print(f"Generator reactive Power = {Qg:.10f} [pu], or {Qg*SgN:.10f} MVAr")

PrintPolar("Vg", Vg, 3, 1)
Sgg = complex(Pg, Qg)
Igg = np.conj(Sgg/Vg)
PrintPolar("Igg", Igg, 3, 1)


The steady-state define the initial conditions for the linear model. The necessary quantities are:

Id = |I|*sin(β)∠(𝛿 - π/2) = 0.617∠-55.7°
Iq = |I|*cos(β)∠𝛿 = 0.646∠34.3°
ΔV_Xd = Xd*Id = 0.648∠34.3°
ΔV_Xq = Xq*Id = 0.426∠124.3°
Eq = 1.48∠34.3°
The internal induced voltage = 14.776 kV
Generator active Power = 0.7991035842 [pu], or 40.0 MW
Generator reactive Power = 0.2365285886 [pu], or 11.8264294277 MVAr
Vg = 0.933∠7.1°
Igg = 0.893∠-9.4°


In [20]:
AB_to_DQ = np.array([               # The rotor angle, 𝛿, is defined w.r.t. to q-axis
    [np.sin(𝛿), -np.cos(𝛿)],
    [np.cos(𝛿), np.sin(𝛿)]
])

I_ab = np.array([
    [I.real],
    [I.imag]
])
print("I_ab = \n",I_ab)

C_dq_current = np.matmul(AB_to_DQ, I_ab)
Id_v = cmath.rect((C_dq_current[0][0]), 𝛿- cmath.pi/2) # 𝛿 - 90°, phi must be in radian
Iq_v = cmath.rect((C_dq_current[1][0]), 𝛿) # 𝛿
print("I_dq = \n",C_dq_current)

PrintPolar("Id", Id_v, 3, 1)
PrintPolar("Iq", Iq_v, 3, 1)

I_ab = 
 [[ 0.88118129]
 [-0.14510307]]
I_dq = 
 [[0.61667951]
 [0.64594253]]
Id = 0.617∠-55.7°
Iq = 0.646∠34.3°


![TET4180 SMIB](delta_conventions_Synchronous_Machine_Modeling_PowerWorld.png)